### Create centroids from polygons for graduated symbol mapping
To do this we'll need to project to polygon back to a projected CRS

In [2]:
# import all the libraries
import numpy as np
import matplotlib as mp
import fiona
import shapely
from shapely.geometry import Polygon, LineString, Point
import pandas as pd
import geopandas as gpd
import rasterio as rio
import rasterstats as rs
import matplotlib.pyplot as plt
from rasterio.plot import show
import mapclassify
from functools import reduce

#change default figure size
plt.rcParams['figure.figsize'] = (12,12)

# show all columns
pd.set_option("display.max_columns", None)

In [3]:
# read in cdps_svis_whp shapefile as dataframe
cdps_svis_whp = gpd.read_file(r'C:\NewMapsPlus\Map698\us-communities-fire\notebooks\data\cdps_svis_whp.shp')

In [4]:
# verify the object type
type(cdps_svis_whp)

geopandas.geodataframe.GeoDataFrame

In [5]:
# get a sample of the data
cdps_svis_whp.sample()

,GEOID,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,AFFGEOID_x,NAME_x,NAMELSAD_x,STUSPS_x,STATE_NAME,Acres_x,whp_score,se_wm,hcd_wm,m_wm,htt_wm,overall_wm,geometry
100,0151456,01,ALABAMA,AL,01073,Jefferson,01073011302,"Census Tract 113.02, Jefferson County, Alabama",19.835812,1600000US0151456,Morris,Morris town,AL,Alabama,2046.095521,3.0,0.368524,0.631279,0.121707,0.17392,0.234538,"POLYGON ((-86.79738 33.72546, -86.79770 33.724..."


#### Project geodataframe for overlay analysis

In [6]:
# check the crs - it's wgs84
cdps_svis_whp.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
# set a variable with parameters for USA Contiguous Equidistant
project_params  = '+proj=eqdc +lat_0=39 +lon_0=-96 +lat_1=33 +lat_2=45 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs'

# reproject and assign to new variable
cdps_svis_whp_ec = cdps_svis_whp.to_crs(project_params)

In [8]:
# # check the crs - it's Equidistant Conic
cdps_svis_whp_ec.crs

<Projected CRS: +proj=eqdc +lat_0=39 +lon_0=-96 +lat_1=33 +lat_2=4 ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Equidistant Conic
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [9]:
# get a sample of the reprojected data
cdps_svis_whp_ec.sample()

,GEOID,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,AFFGEOID_x,NAME_x,NAMELSAD_x,STUSPS_x,STATE_NAME,Acres_x,whp_score,se_wm,hcd_wm,m_wm,htt_wm,overall_wm,geometry
502,0619570,06,CALIFORNIA,CA,06009,Calaveras,06009000504,"Census Tract 5.04, Calaveras County, California",139.248746,1600000US0619570,Dorrington,Dorrington CDP,CA,California,2616.709351,5.0,0.6012,0.554,0.0774,0.5562,0.4683,"POLYGON ((-2088677.571 200893.017, -2088498.79..."


In [10]:
cdps_svis_whp_ctr.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2818 entries, 0 to 2817
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   GEOID       2818 non-null   object  
 1   ST          2818 non-null   object  
 2   STATE       2818 non-null   object  
 3   ST_ABBR     2818 non-null   object  
 4   STCNTY      2818 non-null   object  
 5   COUNTY      2818 non-null   object  
 6   FIPS        2818 non-null   object  
 7   LOCATION    2818 non-null   object  
 8   AREA_SQMI   2818 non-null   float64 
 9   AFFGEOID_x  2818 non-null   object  
 10  NAME_x      2818 non-null   object  
 11  NAMELSAD_x  2818 non-null   object  
 12  STUSPS_x    2818 non-null   object  
 13  STATE_NAME  2818 non-null   object  
 14  Acres_x     2818 non-null   float64 
 15  whp_score   2818 non-null   float64 
 16  se_wm       2818 non-null   float64 
 17  hcd_wm      2818 non-null   float64 
 18  m_wm        2818 non-null   float64 
 19

In [17]:
# change variable name for later use
cdps_svi_whp_ec = cdps_svis_whp_ctr

In [18]:
# convert polygon data to centroid 
cdps_svis_whp_ctr['geometry'] = cdps_svis_whp_ctr['geometry'].centroid

In [19]:
# get a sample of the centroids and notice that the geometry is now point
cdps_svis_whp_ctr.sample()

,GEOID,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,AFFGEOID_x,NAME_x,NAMELSAD_x,STUSPS_x,STATE_NAME,Acres_x,whp_score,se_wm,hcd_wm,m_wm,htt_wm,overall_wm,geometry
1702,3237190,32,NEVADA,NV,32005,Douglas,32005001900,"Census Tract 19, Douglas County, Nevada",56.811046,1600000US3237190,Johnson Lane,Johnson Lane CDP,NV,Nevada,13986.044444,3.0,0.09532,0.293437,0.192435,0.054948,0.043774,POINT (-2019585.500 269590.205)


In [13]:
cdps_svis_whp_ctr.to_file(r'C:\NewMapsPlus\Map698\us-communities-fire\notebooks\data\cdps_svis_whp_ctr.shp')

#### STOPPED HERE 3/30/2022 - NEED TO PLOT CENTROID IN POLYGON

In [ ]:
cdps_svis_whp_ctr.explor